# 🚀 Step 5: Complete LoRA Fine-tuning Pipeline

## Week 7-8: Production-Ready Training System

This is where everything comes together! We'll build a **professional-grade training pipeline** that you could use in real companies.

### 🎯 What You'll Learn:
1. **Professional training loops** - How to structure training for production
2. **Monitoring and metrics** - Track progress like a real ML engineer
3. **Model evaluation** - Comprehensive performance analysis
4. **Comparison baselines** - LoRA vs Full Fine-tuning vs Pre-trained
5. **Deployment preparation** - Making models production-ready

### 🏢 Why This Matters:
This is **exactly** what you'd do as an AI engineer:
- Build robust training pipelines
- Monitor model performance
- Compare different approaches
- Prepare models for deployment
- Document results for stakeholders

In [ ]:
# First, let's import everything we need and set up our environment
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import time
import json
from datetime import datetime
from tqdm.auto import tqdm
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("🚀 Complete Training Pipeline Setup")
print("=" * 50)
print(f"📅 Training started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Project: Email Classification with LoRA Fine-tuning")
print(f"📚 Bootcamp: Week 7-8 - Fine-tuning with LoRA/QLoRA, PEFT")

## 📧 Step 1: Load Data and Model from Previous Step

We'll recreate the essential components from Step 4 to ensure this notebook runs independently.

In [ ]:
# Recreate the email dataset generator from Step 4
class EmailDatasetGenerator:
    def __init__(self):
        self.categories = {
            'urgent': 0, 'support': 1, 'sales': 2, 'spam': 3, 'normal': 4
        }
        
        self.email_templates = {
            'urgent': [
                "URGENT: System is down and affecting all customers. Please respond immediately.",
                "CRITICAL: Security breach detected. Need immediate action from the team.",
                "EMERGENCY: Client meeting in 1 hour, presentation file corrupted. Help needed now!",
                "URGENT RESPONSE NEEDED: Major bug in production causing revenue loss."
            ],
            'support': [
                "Hi, I'm having trouble logging into my account. Can you help me reset my password?",
                "Hello, the software keeps crashing when I try to export data. What should I do?",
                "I can't find the feature you mentioned in the tutorial. Could you guide me?",
                "The mobile app is not syncing with my desktop. How can I fix this?"
            ],
            'sales': [
                "I'm interested in your enterprise plan. Can you send me pricing information?",
                "We're looking for a solution for our team of 50 people. What do you recommend?",
                "Could we schedule a demo to see how your product fits our needs?",
                "I saw your product at the conference. Can we discuss a potential partnership?"
            ],
            'spam': [
                "Congratulations! You've won $1,000,000! Click here to claim your prize now!",
                "AMAZING OFFER: Buy one get one free! Limited time only! Act now!",
                "Your account will be suspended unless you verify immediately. Click this link.",
                "Hot singles in your area want to meet you! Join now for free!"
            ],
            'normal': [
                "Thanks for the meeting yesterday. Here are the notes we discussed.",
                "The project timeline looks good. Let's proceed with the next phase.",
                "I've reviewed the documents and have a few questions. Can we chat tomorrow?",
                "The team meeting is scheduled for Friday at 2 PM in conference room B."
            ]
        }
    
    def generate_email(self, category: str) -> Dict[str, str]:
        body = random.choice(self.email_templates[category])
        subject = f"Re: {body.split('.')[0][:30]}..."
        
        return {
            'subject': subject,
            'body': body,
            'category': category,
            'label': self.categories[category]
        }
    
    def generate_dataset(self, samples_per_category: int = 150) -> pd.DataFrame:
        emails = []
        for category in self.categories.keys():
            for _ in range(samples_per_category):
                email = self.generate_email(category)
                emails.append(email)
        
        random.shuffle(emails)
        return pd.DataFrame(emails)

# Generate dataset
generator = EmailDatasetGenerator()
email_df = generator.generate_dataset(samples_per_category=150)

print(f"📧 Generated dataset: {len(email_df)} emails across {len(generator.categories)} categories")
print(f"   Categories: {list(generator.categories.keys())}")

In [ ]:
# Text preprocessing
class EmailTextPreprocessor:
    def clean_text(self, text: str) -> str:
        if pd.isna(text):
            return ""
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def prepare_for_training(self, df: pd.DataFrame, test_size: float = 0.2) -> Tuple:
        # Clean and combine text
        df['clean_subject'] = df['subject'].apply(self.clean_text)
        df['clean_body'] = df['body'].apply(self.clean_text)
        df['combined_text'] = df['clean_subject'] + ' ' + df['clean_body']
        
        X = df['combined_text'].values
        y = df['label'].values
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )
        
        return X_train, X_test, y_train, y_test, df

# Preprocess data
preprocessor = EmailTextPreprocessor()
X_train, X_test, y_train, y_test, processed_df = preprocessor.prepare_for_training(email_df)

print(f"📊 Data split: {len(X_train)} train, {len(X_test)} test samples")

## 🎯 Step 2: Build LoRA Model and Components

Now let's create our LoRA model and all necessary components for training.

In [ ]:
# LoRA implementation
class AdvancedLoRALayer(nn.Module):
    def __init__(self, original_layer: nn.Linear, rank: int = 4, alpha: float = 32.0, dropout: float = 0.1):
        super().__init__()
        self.original_layer = original_layer
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        
        self.in_features = original_layer.in_features
        self.out_features = original_layer.out_features
        
        self.lora_A = nn.Parameter(torch.empty(rank, self.in_features))
        self.lora_B = nn.Parameter(torch.empty(self.out_features, rank))
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        
        self.reset_lora_parameters()
        
        for param in self.original_layer.parameters():
            param.requires_grad = False
    
    def reset_lora_parameters(self):
        import math
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        result = self.original_layer(x)
        lora_output = F.linear(x, self.lora_A)
        lora_output = self.dropout(lora_output)
        lora_output = F.linear(lora_output, self.lora_B.T)
        result += lora_output * self.scaling
        return result
    
    def get_lora_parameters(self):
        return [self.lora_A, self.lora_B]

# Dataset class
class EmailDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length: int = 128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text, truncation=True, padding='max_length',
            max_length=self.max_length, return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✅ LoRA components defined successfully!")

In [ ]:
# LoRA Email Classifier
class LoRAEmailClassifier(nn.Module):
    def __init__(self, model_name: str = 'distilbert-base-uncased', num_classes: int = 5, 
                 lora_rank: int = 8, lora_alpha: float = 16.0, lora_dropout: float = 0.1):
        super().__init__()
        
        print(f"🏗️  Building LoRA Email Classifier...")
        print(f"   Base model: {model_name}")
        print(f"   Classes: {num_classes}")
        print(f"   LoRA rank: {lora_rank}, alpha: {lora_alpha}")
        
        self.backbone = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # Apply LoRA to attention layers
        self.lora_layers = {}
        self._apply_lora(lora_rank, lora_alpha, lora_dropout)
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.backbone.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
        
        for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
        
        self._print_parameter_stats()
    
    def _apply_lora(self, rank: int, alpha: float, dropout: float):
        target_modules = ['query', 'key', 'value']
        replaced_count = 0
        
        for name, module in self.backbone.named_modules():
            if any(target in name for target in target_modules) and isinstance(module, nn.Linear):
                lora_layer = AdvancedLoRALayer(module, rank=rank, alpha=alpha, dropout=dropout)
                
                parent_module = self.backbone
                module_parts = name.split('.')
                
                for part in module_parts[:-1]:
                    parent_module = getattr(parent_module, part)
                
                setattr(parent_module, module_parts[-1], lora_layer)
                self.lora_layers[name] = lora_layer
                replaced_count += 1
        
        print(f"   ✅ Applied LoRA to {replaced_count} layers")
    
    def _print_parameter_stats(self):
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        print(f"\n📊 Model Statistics:")
        print(f"   Total parameters: {total_params:,}")
        print(f"   Trainable parameters: {trainable_params:,}")
        print(f"   Trainable percentage: {100 * trainable_params / total_params:.2f}%")
        print(f"   Memory reduction: {total_params / trainable_params:.1f}x")
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0]
        logits = self.classifier(cls_output)
        
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)
        
        return {
            'logits': logits,
            'loss': loss,
            'predictions': torch.argmax(logits, dim=-1)
        }
    
    def get_trainable_parameters(self):
        return [p for p in self.parameters() if p.requires_grad]

# Create model
model = LoRAEmailClassifier(
    model_name='distilbert-base-uncased',
    num_classes=5,
    lora_rank=8,
    lora_alpha=16.0
)

print("\n✅ LoRA Email Classifier built successfully!")

## 🚀 Step 3: Professional Training Pipeline

Now let's create our production-ready training system with monitoring, early stopping, and comprehensive logging.

In [ ]:
# Create datasets and dataloaders
train_dataset = EmailDataset(
    texts=X_train.tolist(),
    labels=y_train.tolist(),
    tokenizer=model.tokenizer,
    max_length=128
)

test_dataset = EmailDataset(
    texts=X_test.tolist(),
    labels=y_test.tolist(),
    tokenizer=model.tokenizer,
    max_length=128
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"✅ Datasets created:")
print(f"   Training batches: {len(train_loader)}")
print(f"   Testing batches: {len(test_loader)}")

# Test forward pass
sample_batch = next(iter(train_loader))
model.eval()
with torch.no_grad():
    outputs = model(
        input_ids=sample_batch['input_ids'],
        attention_mask=sample_batch['attention_mask'],
        labels=sample_batch['labels']
    )

print(f"\n✅ Forward pass test successful:")
print(f"   Output logits shape: {outputs['logits'].shape}")
print(f"   Loss: {outputs['loss'].item():.4f}")

In [ ]:
# Professional Training Class
class LoRAEmailTrainer:
    def __init__(self, model, train_loader, val_loader, learning_rate: float = 2e-4,
                 weight_decay: float = 0.01, max_epochs: int = 5, patience: int = 3, device: str = 'cpu'):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.max_epochs = max_epochs
        self.patience = patience
        
        self.trainable_params = model.get_trainable_parameters()
        self.optimizer = torch.optim.AdamW(self.trainable_params, lr=learning_rate, weight_decay=weight_decay)
        
        self.current_epoch = 0
        self.best_val_accuracy = 0.0
        self.best_model_state = None
        self.patience_counter = 0
        self.training_history = {
            'train_loss': [], 'train_accuracy': [],
            'val_loss': [], 'val_accuracy': [],
            'learning_rates': [], 'epoch_times': []
        }
        
        print(f"🔧 Trainer initialized:")
        print(f"   Trainable parameters: {sum(p.numel() for p in self.trainable_params):,}")
        print(f"   Learning rate: {learning_rate}")
        print(f"   Max epochs: {max_epochs}")
        print(f"   Device: {device}")
    
    def train_epoch(self) -> Dict[str, float]:
        self.model.train()
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        pbar = tqdm(self.train_loader, desc=f'Epoch {self.current_epoch + 1}')
        
        for batch_idx, batch in enumerate(pbar):
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            labels = batch['labels'].to(self.device)
            
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs['loss']
            predictions = outputs['predictions']
            
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.trainable_params, max_norm=1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)
            
            current_accuracy = correct_predictions / total_samples
            pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{current_accuracy:.3f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        avg_loss = total_loss / len(self.train_loader)
        accuracy = correct_predictions / total_samples
        
        return {
            'loss': avg_loss,
            'accuracy': accuracy,
            'learning_rate': self.optimizer.param_groups[0]['lr']
        }
    
    def validate(self) -> Dict[str, float]:
        self.model.eval()
        total_loss = 0.0
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc='Validating', leave=False):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)
                
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                
                total_loss += outputs['loss'].item()
                all_predictions.extend(outputs['predictions'].cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.val_loader)
        accuracy = accuracy_score(all_labels, all_predictions)
        
        return {
            'loss': avg_loss,
            'accuracy': accuracy,
            'predictions': all_predictions,
            'labels': all_labels
        }
    
    def train(self) -> Dict:
        print(f"\n🚀 Starting training for {self.max_epochs} epochs...")
        training_start_time = time.time()
        
        for epoch in range(self.max_epochs):
            self.current_epoch = epoch
            epoch_start_time = time.time()
            
            train_metrics = self.train_epoch()
            val_metrics = self.validate()
            
            epoch_time = time.time() - epoch_start_time
            
            self.training_history['train_loss'].append(train_metrics['loss'])
            self.training_history['train_accuracy'].append(train_metrics['accuracy'])
            self.training_history['val_loss'].append(val_metrics['loss'])
            self.training_history['val_accuracy'].append(val_metrics['accuracy'])
            self.training_history['learning_rates'].append(train_metrics['learning_rate'])
            self.training_history['epoch_times'].append(epoch_time)
            
            print(f"\n📈 Epoch {epoch + 1}/{self.max_epochs} Results:")
            print(f"   Train Loss: {train_metrics['loss']:.4f} | Train Acc: {train_metrics['accuracy']:.3f}")
            print(f"   Val Loss: {val_metrics['loss']:.4f} | Val Acc: {val_metrics['accuracy']:.3f}")
            print(f"   Time: {epoch_time:.1f}s")
            
            if val_metrics['accuracy'] > self.best_val_accuracy:
                self.best_val_accuracy = val_metrics['accuracy']
                self.best_model_state = self.model.state_dict().copy()
                self.patience_counter = 0
                print(f"   🎉 New best validation accuracy: {self.best_val_accuracy:.3f}")
            else:
                self.patience_counter += 1
                print(f"   ⏳ No improvement for {self.patience_counter} epochs")
            
            if self.patience_counter >= self.patience:
                print(f"\n🛑 Early stopping triggered after {epoch + 1} epochs")
                break
        
        total_training_time = time.time() - training_start_time
        
        print(f"\n✅ Training completed!")
        print(f"   Total time: {total_training_time:.1f}s")
        print(f"   Best validation accuracy: {self.best_val_accuracy:.3f}")
        
        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
            print(f"   📥 Loaded best model weights")
        
        return {
            'best_val_accuracy': self.best_val_accuracy,
            'total_time': total_training_time,
            'epochs_trained': epoch + 1,
            'training_history': self.training_history
        }

print("✅ Professional Training Class Ready!")

In [ ]:
# Setup device and create trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")

trainer = LoRAEmailTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,
    learning_rate=2e-4,
    weight_decay=0.01,
    max_epochs=5,
    patience=3,
    device=device
)

print(f"\n🎯 Training Configuration:")
print(f"   Model: DistilBERT + LoRA (rank=8)")
print(f"   Dataset: {len(X_train)} train, {len(X_test)} test")
print(f"   Batch size: 16 (train), 32 (test)")
print(f"   Learning rate: 2e-4")
print(f"   Early stopping: 3 epochs patience")

# Train the model
print(f"\n🚀 Let's train our LoRA email classifier!")
training_results = trainer.train()

## 📊 Step 4: Comprehensive Model Evaluation

Now let's thoroughly evaluate our trained model with professional-grade metrics and analysis.

In [ ]:
def comprehensive_model_evaluation(model, test_loader, category_mapping, device):
    print("🔬 Comprehensive Model Evaluation")
    print("=" * 50)
    
    model.eval()
    all_predictions = []
    all_labels = []
    all_probabilities = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            
            probabilities = F.softmax(outputs['logits'], dim=-1)
            predictions = torch.argmax(outputs['logits'], dim=-1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
    
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    all_probabilities = np.array(all_probabilities)
    
    category_names = list(category_mapping.keys())
    overall_accuracy = accuracy_score(all_labels, all_predictions)
    
    print(f"\n🎯 Overall Accuracy: {overall_accuracy:.3f}")
    
    print(f"\n📊 Detailed Classification Report:")
    report = classification_report(all_labels, all_predictions, target_names=category_names, digits=3)
    print(report)
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=category_names, yticklabels=category_names, ax=axes[0, 0])
    axes[0, 0].set_title('Confusion Matrix')
    axes[0, 0].set_xlabel('Predicted')
    axes[0, 0].set_ylabel('Actual')
    
    # Per-class metrics
    from sklearn.metrics import precision_recall_fscore_support
    precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_predictions, average=None)
    
    x = np.arange(len(category_names))
    width = 0.25
    
    axes[0, 1].bar(x - width, precision, width, label='Precision', alpha=0.8)
    axes[0, 1].bar(x, recall, width, label='Recall', alpha=0.8)
    axes[0, 1].bar(x + width, f1, width, label='F1-Score', alpha=0.8)
    
    axes[0, 1].set_xlabel('Categories')
    axes[0, 1].set_ylabel('Score')
    axes[0, 1].set_title('Per-Class Performance Metrics')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(category_names, rotation=45)
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Training curves
    epochs = range(1, len(training_results['training_history']['train_loss']) + 1)
    axes[1, 0].plot(epochs, training_results['training_history']['train_loss'], 'b-', label='Train Loss')
    axes[1, 0].plot(epochs, training_results['training_history']['val_loss'], 'r-', label='Val Loss')
    axes[1, 0].set_title('Training Loss Curves')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Accuracy curves
    axes[1, 1].plot(epochs, training_results['training_history']['train_accuracy'], 'b-', label='Train Acc')
    axes[1, 1].plot(epochs, training_results['training_history']['val_accuracy'], 'r-', label='Val Acc')
    axes[1, 1].set_title('Training Accuracy Curves')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'accuracy': overall_accuracy,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities,
        'confusion_matrix': cm,
        'classification_report': report
    }

# Perform evaluation
eval_results = comprehensive_model_evaluation(
    model=model,
    test_loader=test_loader,
    category_mapping=generator.categories,
    device=device
)

## 🎯 Step 5: Real-World Demo

Let's test our model with new emails to see how it performs in practice!

In [ ]:
def create_email_classifier_demo(model, tokenizer, category_mapping, device):
    def classify_email(subject: str, body: str):
        # Preprocess
        preprocessor = EmailTextPreprocessor()
        clean_subject = preprocessor.clean_text(subject)
        clean_body = preprocessor.clean_text(body)
        combined_text = clean_subject + ' ' + clean_body
        
        # Tokenize
        inputs = tokenizer(
            combined_text, truncation=True, padding='max_length',
            max_length=128, return_tensors='pt'
        )
        
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        
        model.eval()
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probabilities = F.softmax(outputs['logits'], dim=-1)
            predicted_class = torch.argmax(outputs['logits'], dim=-1)
        
        predicted_label = predicted_class.item()
        predicted_category = list(category_mapping.keys())[list(category_mapping.values()).index(predicted_label)]
        confidence = probabilities[0][predicted_label].item()
        
        # All probabilities
        prob_dict = {}
        for category, label in category_mapping.items():
            prob_dict[category] = probabilities[0][label].item()
        
        return {
            'predicted_category': predicted_category,
            'confidence': confidence,
            'all_probabilities': prob_dict
        }
    
    return classify_email

# Create demo
email_classifier = create_email_classifier_demo(
    model=model, tokenizer=model.tokenizer,
    category_mapping=generator.categories, device=device
)

# Test emails
test_emails = [
    {
        'subject': 'URGENT: Server Down - Revenue Impact!',
        'body': 'The main server crashed and we are losing money every minute. Please fix this immediately!',
        'expected': 'urgent'
    },
    {
        'subject': 'Question about your pricing plans',
        'body': 'Hi, I am interested in your enterprise plan for my company of 100 employees. Could you send me a quote?',
        'expected': 'sales'
    },
    {
        'subject': 'You have won $1 million dollars!!!',
        'body': 'Congratulations! You are our lucky winner! Click here now to claim your amazing prize!',
        'expected': 'spam'
    }
]

print("🧪 Testing with New Emails:")
print("=" * 50)

correct_predictions = 0
for i, email in enumerate(test_emails):
    print(f"\n📧 Test Email {i+1}:")
    print(f"   Subject: {email['subject']}")
    print(f"   Expected: {email['expected']}")
    
    result = email_classifier(email['subject'], email['body'])
    
    print(f"   Predicted: {result['predicted_category']} (confidence: {result['confidence']:.3f})")
    
    is_correct = result['predicted_category'] == email['expected']
    if is_correct:
        correct_predictions += 1
        print(f"   Result: ✅ CORRECT")
    else:
        print(f"   Result: ❌ INCORRECT")
    
    print(f"   All probabilities:")
    for category, prob in result['all_probabilities'].items():
        marker = "👉" if category == result['predicted_category'] else "  "
        print(f"      {marker} {category}: {prob:.3f}")

demo_accuracy = correct_predictions / len(test_emails)
print(f"\n🎯 Demo Accuracy: {correct_predictions}/{len(test_emails)} = {demo_accuracy:.3f}")
print(f"✅ Model is ready for deployment!")

## 🏆 Training Summary

Congratulations! You've successfully built and trained a production-ready LoRA email classifier. Here's what you've accomplished:

### ✅ **Key Achievements:**
- **Built a complete training pipeline** with monitoring and early stopping
- **Achieved strong performance** with minimal trainable parameters
- **Created professional evaluation metrics** and visualizations
- **Demonstrated real-world applicability** with practical examples
- **Implemented best practices** for production ML systems

### 🎯 **Next Steps:**
In Step 6, we'll compare our approach with alternatives and create a comprehensive business analysis.

You're now ready to deploy this system or adapt it for other text classification tasks!